# Stock Price Prediction with LSTM (Long Short-Term Memory) Neural Network 

In [1]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta

In [11]:
today = date.today()

d1 = today.strftime('%Y-%m-%d')
end_date = d1

d2 = date.today() - timedelta(days=7000)
d2 = d2.strftime('%Y-%m-%d')
start_date = d2

# print(start_date)
data = yf.download('AAPL',
                   start=start_date,
                   end=end_date,
                   progress=False,
                   auto_adjust=False)

# data = data.reset_index()
# print(data)

# data = [['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
# data.columns = ['date', 'open', 'high', 'low', 'close', 'adj close', 'volume']
# print(data)

data['Date'] = data.index

data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
data.reset_index(drop=True, inplace=True)
data.columns = data.columns.get_level_values(0)

In [12]:
print(data.tail())

Price       Date        Open        High         Low       Close   Adj Close  \
4816  2025-09-18  239.970001  241.199997  236.649994  237.880005  237.880005   
4817  2025-09-19  241.229996  246.300003  240.210007  245.500000  245.500000   
4818  2025-09-22  248.300003  256.640015  248.119995  256.079987  256.079987   
4819  2025-09-23  255.880005  257.339996  253.580002  254.429993  254.429993   
4820  2025-09-24  255.220001  255.740005  251.039993  252.309998  252.309998   

Price     Volume  
4816    44249600  
4817   163741300  
4818   105517400  
4819    60275200  
4820    42265200  


In [13]:
import plotly.graph_objects as go


figure = go.Figure(data=[go.Candlestick(x = data['Date'],
                                        open = data['Open'],
                                        high = data['High'],
                                        low = data['Low'],
                                        close = data['Close'])])

figure.show()

In [14]:
correlation = data.corr()
print(correlation['Close'].sort_values(ascending=False))

Price
Close        1.000000
Adj Close    0.999915
High         0.999911
Low          0.999906
Open         0.999804
Date         0.884712
Volume      -0.541705
Name: Close, dtype: float64


In [26]:
x = data[['Open', 'High', 'Low', 'Volume']]
x = x.to_numpy()

y = data[['Close']]
y = y.to_numpy()
# print(type(y))
y = y.reshape(-1, 1)
# print(type(y))

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

In [29]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))

model.add(Dense(25))
model.add(Dense(1))

model.summary()

/run/media/ooka/cae585b1-9ae2-453d-9643-e7f830589189/jinx/Belajar/ML Projects/.venv/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 4, 128)         │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=30)

Epoch 1/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 439.9812
Epoch 2/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - loss: 25.4978
Epoch 3/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 23.2453
Epoch 4/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 33.8995
Epoch 5/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 17.1112
Epoch 6/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 12.1986
Epoch 7/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 10.2051
Epoch 8/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 12.8298
Epoch 9/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 18.5384
Epoch 10/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 12.3518
Epoch 11/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 10.1054
Epoch 12/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 10.1171
Epoch 13/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 11.5920
Epoch 14/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 21.2174
Epoch 15/30


In [32]:
import numpy as np
# features = Open, High, Low, Low, Adj Close, Volume
fitur = np.array([[180.086665, 185.980003, 178.330990, 82983342]])
model.predict(fitur)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step


array([[179.587]], dtype=float32)